# A Test for the Cytoscape Example

In [ ]:
import json
import os
import time
from pathlib import Path

import importnb
from rdflib import Graph
from requests_cache import CachedSession

with importnb.Notebook():
    try:
        from .Cytoscape_Example import cy, pl, W, cy_ex, fm
    except ImportError:
        from Cytoscape_Example import cy, pl, W, cy_ex, fm

This notebook is used as part of the `ipyradiant` test suite, where `IPYRADIANT_TESTING`
will be set, which will trigger the automted section below.

In [ ]:
IS_TESTING = json.loads(os.environ.get("IPYRADIANT_TESTING", "false"))
IS_TESTING

Example data is loaded from [schema.org](https://schema.org/docs/developers.html).

Tests are implemented as buttons you click.

In [ ]:
tests = [
    W.Button(description=f"{k}")
    for k, v in pl.file_picker.options.items()
    if v and "tree" not in k
]

In [ ]:
timings = {}


def timestamp(key, msg):
    if key not in timings:
        timings[key] = []
    timings[key] += [time.time()]
    delta = 0 if len(timings[key]) == 1 else timings[key][-1] - timings[key][-2]
    print(f"[{key}]", f"+{int(delta)}", msg)

In [ ]:
def _run_test(btn):
    p = btn.description

    try:
        timestamp(p, "starting...")
        fm.graph = Graph()
        timestamp(p, "cleaned...")
        assert len(cy.graph.all_nodes()) == 0
        timestamp(p, f"loading...")
        pl.file_picker.value = pl.file_picker.options[p]
        timestamp(p, f"... {len(fm.graph)} triples loaded")
        assert len(fm.graph)
        assert len(cy.graph.all_nodes()) > 0
        timestamp(p, "OK!")
    except Exception as err:
        timestamp(p, "ERROR")
        timestamp(p, err)
        raise Exception(f"{p} failed") from err


[d.on_click(_run_test) for d in tests]

## Show the Test Application

In [ ]:
W.VBox([cy_ex, W.HBox([W.Label("Start Test"), *tests])])

# Run Test Automatically

In [ ]:
if IS_TESTING:
    for test in tests:
        test.click()